In [ ]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize
import datetime
from lxml import html
import os 
import csv
from random import randint

On récupère les choix utilisateurs

In [38]:
print("\n \t $$$$$$$$$$$$$$\n\n      $$$  FINANCE APP     §§§\n \n \t $$$$$$$$$$$$$$\n \n ")


 	 $$$$$$$$$$$$$$

      $$$  FINANCE APP     §§§
 
 	 $$$$$$$$$$$$$$
 
 


In [44]:
choix = input("\n 1 - SOPRA\n 2 - L'OREAL\n 3 - AIR LIQUIDE\n 4 - BIO-MERIEUX\n 5 - TECHNIP\n 6 - ATOS\n 7 - Quitter\n \n  Choisissez un titre: ")


 1 - SOPRA
 2 - L'OREAL
 3 - AIR LIQUIDE
 4 - BIO-MERIEUX
 5 - TECHNIP
 6 - ATOS
 7 - Quitter
 
  Choisissez un titre: 6


On choisi un titre en fonction du choix de l'utilisateur

In [40]:
if choix == '1':
    titre = "SOP.PA"
    url = "https://www.boursorama.com/cours/1rPSOP/"
    action = "sopra"
elif choix == '2':
    titre = "OR.PA"
    url = "https://www.boursorama.com/cours/1rPOR/"
    action = "loreal"
elif choix == '3':
    titre = "AI.PA"
    url = "https://www.boursorama.com/cours/1rPAI/"
    action = "airliquide"
elif choix == '4':
    titre = "BIM.PA"
    url = "https://www.boursorama.com/cours/1rPBIM/"
    action = "biomerieux"
elif choix == '5':
    titre = "FTI.PA"
    url = "https://www.boursorama.com/cours/1rPAI/"
    action = "technipfmc"
elif choix == '6':
    titre = "ATO.PA"
    url = "https://www.boursorama.com/cours/1rPATO/"
    action = "atos"
    
elif choix == '7':
    exit()
else:
    print("Le titre choisi n'est pas pris en compte. \n ")
    exit()
    

print(titre)

AI.PA


Fonction pour changer le format des dates de AAAA-MM-JJ HH:MM:SS -> AAAA-MM-JJ HH:MM

In [41]:
def reformat_date(index):
    date = datetime.datetime.strptime(index, '%Y-%m-%d %H:%M:%S')
    date = datetime.datetime.strftime(date, '%Y-%m-%d %H:%M')
    
    return date

On définit la fonction qui chargera les données pour un indicateur technique

In [42]:
def analyse(titre ,fonction):
    periode = "20"
    
    #Les différentes parties de l'url
    part1_url_analyse = "https://www.alphavantage.co/query?function="
    part2_url_analyse = "&symbol="
    part3_url_analyse = "&interval=1min&time_period="
    part4_url_analyse = "&series_type=close&apikey=2FWTNAS6MFKAV1XH"
    
    #On construit l'url
    url_analyse = part1_url_analyse + fonction + part2_url_analyse + titre + part3_url_analyse + periode + part4_url_analyse
    
    #On exécute la requete
    response_an = requests.get(url_analyse)
    
    #On convertit le résultat de la requete en json puis en dataFrame
    data_an = json.loads(response_an.content)
    analyses = pd.DataFrame.from_dict(data_an["Technical Analysis: "+fonction], orient='index')
    
    return analyses
    

In [7]:
titre

'FTI.PA'

FONTION POUR OBTENIR LES INDICATEURS TECHNIQUES

In [8]:
def analyse_tech(titre):
    #On construit l'url de la requete API avec une concaténation de chaines de caractères
    
    #Mais avant on prépare les parties de l'url
    part1_URL = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol="
    part2_URL = "&interval=1min&outputsize=full&apikey=2FWTNAS6MFKAV1XH"
    
    #Maintenant on construit l'url pour l'action en bourse 
    url = part1_URL + titre + part2_URL
    
    #On exécute la requête
    response = requests.get(url)
    
    #On convertit la réponse au format json
    data = json.loads(response.content)
    
    #On convertit en dataFrame
    df_data = pd.DataFrame.from_dict(data['Time Series (1min)'], orient='index')
    
    #On change les index des dataframe au format AAAA-MM-JJ HH:MM
    df_data.index = list(map(reformat_date, df_data.index))
    
    #Liste qui va contenir les fonctions à analyser
    fonctions = ['BBANDS', 'RSI', 'MACD', 'STOCH']
    
    #On calcule tous les indicateurs techniques
    for fonction in fonctions:
        analyse_tech = analyse(titre, fonction)
    
        #on joint le resultat au dataframe
        df_data = pd.concat([df_data, analyse_tech], axis=1, join_axes=[analyse_tech.index])
        
    return df_data

On appelle la fonction pour l'analyse technique

In [9]:
try:
    df_data = analyse_tech(titre)
except:
    print("\n La connexion semble lente, veuillez vérifier votre connexion internet :) ! \n")
    exit()

Fontion pour obtenir les indicateurs financiers

In [10]:
def analyse_fin(url):
    
    analyse_fin = []
    
    #recupere le PER
    page = requests.get(url)
    tree = html.fromstring(page.content)
    per = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[4]/td[2]/text()')
    s= per[0].split('        ')
    b = s[2][0:-1]
    analyse_fin.append(float(b))
    
    #Repcupere le dividende par action
    div = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[1]/td[4]/text()')
    div = div[0].split('        ')
    div = div[2][0:-1]
    analyse_fin.append(float(div))
    
    #recupere le BNA
    bna = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[3]/td[3]/text()')
    bna = bna[0].split('        ')
    bna = bna[2][0:-1]
    analyse_fin.append(float(bna))
    
    #recupere le rendement
    rd = tree.xpath('//*[@id="main-content"]/div/section/div[2]/article/div[2]/div[7]/div[2]/div[1]/div/table/tbody/tr[2]/td[3]/text()')
    rd = rd[0].split('        ')
    rd = rd[2][0:-2]
    analyse_fin.append(float(rd))
    
    return analyse_fin
    
    
    
    
    
    
    

On charge les indicateurs financiers dans une liste dans cet ordre : -------
    1- PER , 
    2- Dividende , 
    3- BNA , 
    4- Rendement , 

In [11]:
finance = analyse_fin(url)

In [12]:
finance

[20.01, 2.95, 5.19, 2.69]

On supprime les NA

In [13]:
df_data = df_data.dropna()

In [14]:
df_data

,1. open,2. high,3. low,4. close,5. volume,Real Upper Band,Real Lower Band,Real Middle Band,RSI,MACD_Signal,MACD_Hist,MACD,SlowK,SlowD
2018-11-26 03:43,20.3700,20.3700,20.3700,20.3700,2156,20.4517,20.3013,20.3765,56.2956,0.0294,-0.0128,0.0165,60.6065,39.3944
2018-11-26 03:45,20.3800,20.3800,20.3800,20.3800,1136,20.4511,20.3069,20.3790,57.0108,0.0267,-0.0105,0.0162,81.8184,59.5964
2018-11-26 03:47,20.3900,20.3900,20.3900,20.3900,1777,20.4494,20.3156,20.3825,57.7388,0.0247,-0.0081,0.0166,100.0000,80.8083
2018-11-26 03:48,20.4000,20.4000,20.4000,20.4000,1618,20.4498,20.3212,20.3855,58.4790,0.0233,-0.0058,0.0175,100.0000,93.9395
2018-11-26 03:50,20.4100,20.4100,20.4100,20.4100,1057,20.4506,20.3274,20.3890,59.2306,0.0224,-0.0036,0.0188,100.0000,100.0000
2018-11-26 03:51,20.3900,20.3900,20.3900,20.3900,1885,20.4510,20.3290,20.3900,57.0562,0.0215,-0.0035,0.0180,77.7778,92.5926
2018-11-26 03:54,20.4000,20.4000,20.4000,20.4000,392,20.4520,20.3300,20.3910,57.8702,0.0208,-0.0028,0.0180,61.1111,79.6296
2018-11-26 03:56,20.3800,20.3800,20.3800,20.3800,1521,20.4486,20.3294,20.3890,55.6494,0.0199,-0.0037,0.0161,27.7778,55.5556
2018-11-26 03:59,20.3900,20.3900,20.3900,20.3900,848,20.4479,20.3291,20.3885,56.5275,0.0189,-0.0036,0.0153,27.7778,38.8889
2018-11-26 04:01,20.4100,20.4100,20.4100,20.4100,1125,20.4465,20.3295,20.3880,58.2669,0.0184,-0.0023,0.0161,44.4444,33.3333


CALCUL DE LA NOTE DE L'ANALYSE TECHNIQUE ( sur 10 )

Fonction pour la note du RSI

In [15]:
def calcul_rsi(rsi):
    if( rsi < 30 ):
        note_rsi = 10 
    elif( 30 <= rsi <= 70 ):
        note_rsi = 5
    else:
        note_rsi = 0
        
    return note_rsi

Fonction pour la note du macd

In [16]:
def calcul_macd(macd):
    
    if (macd > 0):
        note_macd = 10
    elif(macd < 0):
        note_macd = 0
    else:
        note_macd = 5
        
    return note_macd

Fonction pour la note du stockastique

In [17]:
def calcul_stoch(stoch):
    if(stoch < 20 ):
        note_st = 10
    elif(20 <= stoch <80 ):
        note_st = 5
    else:
        note_st = 0
        
    return note_st

Fonction qui calcule la note technique

In [18]:
def calcultech(rsi, macd, sto):
    note_tech = 0.35*5 + 0.25*calcul_rsi(float(rsi)) + 0.25*calcul_macd(float(macd)) + 0.15 *calcul_stoch(float(sto))
    
    return note_tech

On calcule la note technique

In [19]:
note_tech = calcultech(df_data['RSI'][-1], df_data['MACD'][-1], df_data['SlowK'][-1])

In [20]:
note_tech

3.75

Fonction pour evaluer l'analyse financiere

In [21]:
def note_fin(liste):
    if liste[0] <= 10:
        res=0
    elif liste[0] >= 17:
        res=5
    else:
        res=10
    if liste[1] >= 9:
        res+=10
    elif liste[1] >= 7:
        res+=9
    elif liste[1] >= 5:
        res+=8
    elif liste[1] >= 3:
        res+=7
    elif liste[1] >= 2:
        res+=6
    elif liste[1] >= 1:
        res+=5
    else:
        res+=3
    if liste[2] >= 5:
        res+=10
    elif liste[2] >= 3:
        res+=8
    elif liste[2] >= 2:
        res+=7
    elif liste[2] >= 1:
        res+=5
    else:
        res+=3
    if liste[3] >= 4:
        res+=10
    elif liste[3] >= 2:
        res+=5
    else:
        res+=0
    return float(res/4)

On prends la note financiere

In [22]:
note_f = note_fin(finance) 

In [23]:
note_f 

6.5

Fonction qui gère l'information en temps réel

In [45]:
def getInfo(action):
    cpt=0
    filename=os.environ["PUBLIC"]+'\\'+action+'Info.csv'
    with open(filename, 'r') as textfile:
        for row in reversed(list(csv.DictReader(textfile, fieldnames = ("date", "info", "pertinence", "note"), delimiter = ';'))):
            date = row['date']
            if date == "date":
                break
            info = row['info']
            pertinence = row['pertinence']
            note = row['note']
            datetime.datetime.strptime(row['date'], "%Y-%m-%d %H:%M")
            limit = datetime.datetime.now()
            limit = limit - datetime.timedelta(hours=2)
            if datetime.datetime.strptime(row['date'], "%Y-%m-%d %H:%M") < limit:
                if cpt==0:
                    tab=[date,info,pertinence,note]
                    cpt=1
                else:
                    tab=tab,[date,info,pertinence,note]
    cpt = 0
    res=[]
    res.append(0)
    res.append('')
    for i in tab:
        res[0]+=5 + float(i[3]) * float(i[2])
        res[1]+=i[1]+'\n'
        cpt+=1
    res[0]=res[0]/cpt
    
    return res

In [46]:
resultat_info = getInfo(action)

In [47]:
note_info = resultat_info[0]
note_info

1.5

On genere le hazard

In [27]:
hazard = randint(1,8)
hazard

1

on calcule la note finale

In [28]:
note_finale = 0.23 * note_f + 0.20 * note_tech + 0.5 * note_info + 0.07 * hazard
note_finale

3.065

On emet une ecommandation en fonction de la note finale

In [29]:
if(note_finale < 3.33):
    recom = "VENDEZ !!!!"
elif( 3.33 <= note_finale < 7):
    recom = "NE FAITES RIEN! ATTENDEZ !!!"
else:
    recom = "ACHETEZ !!! "

On affiche la recommandation à l'utilisateur et la raison

In [36]:
print("\n ----------------")
print("\n Notre note globale: ")
print(note_finale)
print("\n Détail des notes :\n Note tech : "+ str(note_tech) + "  Note fin : "+ str(note_f) + "  Note info : "+ str(note_info) )
print("\n Notre conseil :" + recom)
print("\n \n RAISON: "+ resultat_info[1])


 ----------------

 Notre note globale: 
3.065

 Détail des notes :
 Note tech : 3.75  Note fin : 6.5  Note info : 1.5

 Notre conseil :VENDEZ !!!!

 
 RAISON: la rupture des 20.8 se confirme
l'action passe sous le plancher a 20.26 risque de ne pas se relever

